In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/main/'

# Which calibration type to analyse.
calibType = 'ptc'
exposureNumber = 2022021800078

# Which camera the calibration is for.
cameraName = 'LSSTComCam'

# Which display to use.
displayBackend = 'matplotlib'

# Which collection the calibration was constructed in.
genCollection = 'u/plazas/2022AUG04-ptc.comcam.FULLCOV.1'

# Which collection containing the verification outputs.
verifyCollection = 'u/plazas/DM-30171.PTC-verify-comcam.2022AUG04.2'

In [ ]:
# Get butler and camera
# the genCollection has the camera
butler = dB.Butler(repository, collections=[genCollection,verifyCollection])
camera = butler.get('camera', instrument=cameraName)
numberDetectors = len(camera)

In [ ]:
# Get Run Statistics
runStats = butler.get('verifyPtcStats', instrument=cameraName)
runSuccess = runStats.pop('SUCCESS')

In [ ]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)

In [ ]:
# Noise failed, let's look at the values
gainThreshold = 5  # percentage
noiseThreshold = 5 # percentage
for detector in camera:
    detId = detector.getId()
    calib = butler.get("ptc", instrument=cameraName, detector=detId)
    detector = camera[detId]
    print (f"Detector ID: {detId}")
    for amp in detector:
        ampName = amp.getName()
        diffGain = np.abs(calib.gain[ampName] - amp.getGain()) / amp.getGain()
        diffNoise = np.abs(calib.noise[ampName] - amp.getReadNoise()) / amp.getReadNoise()
        testGain = bool(diffGain < gainThreshold)
        testNoise = bool(diffNoise < noiseThreshold)
        if testNoise:
            print ('FAILURE NOISE:', ampName, diffNoise , calib.noise[ampName], amp.getReadNoise())
    print(" ")

In [ ]:
# Gain, noise, turnoff, and a00 values are saved as 'AMP' statistics
# The noise values from the fit are not very good, providing insight into why the NOISE 
# tests failed. 
for detId in range(numberDetectors):
    detStats = butler.get("verifyPtcDetStats", instrument=cameraName, detector=detId)
    print (detId, detStats)

In [ ]:
# Plot PTCs
if interactive:
    for detId in range(numberDetectors): 
        ptc = butler.get('ptc', instrument=cameraName, detector=detId)
        continueDisplay=True
        for amp in np.unique(ptc.ampNames):
            plt.scatter(ptc.rawMeans[amp], ptc.rawVars[amp], marker='o', label="PTC-raw")
            plt.scatter(ptc.finalMeans[amp], ptc.finalVars[amp], marker='+', label="PTC")
            plt.show()
            continueDisplay, skipNumber = utils.interactiveBlock(f"{exposureId} {detector.getName()}", 
                                                                 detStats)
            if continueDisplay is False:
                break
    if continueDisplay is False:
        break